In [4]:
# Création d'un objet BeautifulSoup relié à la page web des tests de jeux sur metacritic

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

req = Request("https://www.metacritic.com/browse/games/score/metascore/all/all/filtered", headers = {'User-Agent': 'Mozilla/5.0'})

soup = BeautifulSoup(urlopen(req).read(), 'html.parser')

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1125)>

In [6]:
# Lecture du nombre de pages
tot_pages = soup.find(attrs={'class' : 'page last_page'}).a.text
tot_pages


'188'

In [ ]:
# Scrapping des données

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

game = []
platform = []
rating = []
date = []

# Le scrapping ne peut pas être fait en une seule fois sans donner d'erreurs de timeout. 
# Ici il a été fait par tranches de 25 pages.
for pa in range(0,25):
    url = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page=%d" % pa
    req = Request(url, headers = {'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(urlopen(req).read(), 'html.parser')
    
    games = soup.find_all(name = 'a', attrs = {'class' : 'title'})
    platforms = soup.find_all(name = 'span', attrs = {'class' : 'data'})
    ratings = soup.find_all(name = 'div', attrs = {'class' : re.compile('metascore_w large game*')})
    dates = soup.find_all(attrs = {'class' : 'clamp-details'})
#    print(pa)
#    print(len(games))
    for g, p, r, d in zip(games, platforms, ratings, dates):
        game.append(g.text)
        platform.append(p.text.replace('\n', '').replace('                                        ', '').replace('                                    ', ''))
        rating.append(r.text)
        date.append(d.text[-5:-1])


In [36]:
# Création de la dataframe contenant les données scrappées
df = pd.DataFrame({'game' : game, 'platform' : platform, 'rating' : rating, 'date' : date})

In [37]:
# Exportation dans un fichier csv
df.to_csv('metacritic_date_0-25.csv')